In [7]:
# %run "Imports.ipynb"

In [11]:
# Read in any matchup file to identify columns
test_path = r"C:\Users\james\Documents\MLB\Database\B01. Matchups\Matchups 20240328\BOS@SEA 745283 2210.xlsx"

In [31]:
# Read in batters
batter_df = pd.read_excel(test_path, sheet_name = "AwayBatters")

# Extract column names
batter_columns = batter_df.columns.tolist()

# Read in pitchers
pitcher_df = pd.read_excel(test_path, sheet_name = "AwayPitchers")

# Extract column names
pitcher_columns = pitcher_df.columns.tolist()

del batter_df, pitcher_df

In [34]:
class Batter:
    def __init__(self, **kwargs):
        for column in batter_columns + ['confirmed']:
            setattr(self, column, kwargs.get(column, None))
            
        # DK stats    
        self.HBP = 0
        self.BB = 0
        self.B1 = 0
        self.B2 = 0
        self.B3 = 0
        self.HR = 0
        self.SB = 0
        self.R = 0
        self.RBI = 0
        self.FP = 0

        # Other
        self.PA = 0
        
        # Pitcher that allowed batter to reach
        # Will be none for anyone reaching on error or that would otherwise be unearned
        self.pitcher = None
        # Unearned runs scored
        # When a runner reaches on an error, we don't want to charge the pitcher with their ER, so we just assign it to the batter (where it won't matter)
        self.ER = 0
        # Whether the run is charged to the pitcher or not
        self.charged = 1
        
    def keep_selected_attributes(self):
        # List of attributes to keep
        keep_attributes = ['PA', 'HBP', 'BB', 'B1', 'B2', 'B3', 'HR', 'SB', 'R', 'RBI', 'FP', 'fullName', 'id', 'imp_b_l', 'imp_b_r', 'batting_order', 'confirmed']

        # Remove attributes not in the keep list
        for attr in list(self.__dict__.keys()):
            if attr not in keep_attributes or attr == '__class__':
                self.__dict__.pop(attr, None)

In [36]:
class Pitcher:
    def __init__(self, **kwargs):
        for column in pitcher_columns + ['confirmed']:
            setattr(self, column, kwargs.get(column, None))
            
        # DK stats
        self.winning = False
        self.OUT = 0
        self.SO = 0
        self.HBP = 0
        self.BB = 0
        self.B1 = 0
        self.B2 = 0
        self.B3 = 0
        self.HR = 0
        self.H = 0
        self.CS = 0
        self.R = 0
        self.ER = 0
        self.W = 0
        self.CG = 0
        self.CGSO = 0
        self.NH = 0
        self.FP = 0
        self.PA = 0
        
        self.PO = 0
        self.GO = 0
        self.LO = 0
        self.FO = 0
        
        self.faced = 0
        self.reached = 0
        self.TB = 0
        
        self.SO_inning = 0
        self.HBP_inning = 0
        self.HBP_inning = 0
        self.BB_inning = 0
        self.B1_inning = 0
        self.B2_inning = 0
        self.B3_inning = 0
        self.HR_inning = 0
        self.H_inning = 0
        
        self.PO_inning = 0
        self.GO_inning = 0
        self.LO_inning = 0
        self.FO_inning = 0

        self.ER_inning = 0
        self.faced_inning = 0
        self.reached_inning = 0
        self.TB_inning = 0
        self.OUT_inning = 0 

        self.last_PA = None

        self.imp_p_either = max(self.imp_p_l, self.imp_p_r)
        self.starter = 0
        
    def keep_selected_attributes(self):
        # List of attributes to keep
        keep_attributes = ['PA', 'OUT', 'SO', 'HBP', 'BB', 'B1', 'B2', 'B3', 'HR', 'H', 'R', 'ER', 'W', 'CG', 'CGSO', 'NH', 'FP', 'fullName', 'id', 'imp_p_l', 'imp_p_r', 'leverage', 'confirmed']

        # Remove attributes not in the keep list
        for attr in list(self.__dict__.keys()):
            if attr not in keep_attributes or attr == '__class__':
                self.__dict__.pop(attr, None)

In [1]:
class Scoreboard:
    def __init__(self, away_batters, home_batters, away_pitchers, home_pitchers, innings):
        # Player objects
        self.away_batters = away_batters
        self.home_batters = home_batters
        self.away_pitchers = away_pitchers
        self.home_pitchers = home_pitchers
        
        # Score
        self.away_score = 0
        self.home_score = 0
        self.winning_team = "Tie"
        
        # Inning state
        self.innings = innings
        self.inning = 1
        self.top_bot = "Top"
        self.outs = 0
        self.faced_inning = 0 # deprecated
        self.br_inning = 0 # deprecated
        self.error_extended = False
        
        # Current player vs player matchup
        self.ab = None
        self.pitching = None
        
        # Base states
        # Player
        self.on_1b = None
        self.on_2b = None
        self.on_3b = None
        # Binary
        self.onFirst = 0
        self.onSecond = 0
        self.onThird = 0

        # Batter order
        self.away_order = 1
        self.home_order = 1
        
        # Pitcher leverage
        self.away_leverage = 1
        self.home_leverage = 1
        # Pitchers currently up
        self.away_pitcher_up = None
        self.home_pitcher_up = None
        # Starters have been pulled
        self.away_starter_pulled = False
        self.home_starter_pulled = False
        # Starters
        self.away_starter = None
        self.home_starter = None
        # Pitcher currently in line for win
        self.winning_pitcher = None
        # Zombies
        self.away_zombie = None
        self.home_zombie = None
        
        
        # Create venue binaries
        for venue in venue_inputs:
            setattr(self, f'{venue}', 0)
            
    def keep_selected_attributes(self):
        # List of attributes to keep
        keep_attributes = ['away_batters', 'home_batters', 'away_pitchers', 'home_pitchers', 'away_score', 'home_score', 'innings']

        # Remove attributes not in the keep list
        for attr in list(self.__dict__.keys()):
            if attr not in keep_attributes or attr == '__class__':
                self.__dict__.pop(attr, None)

In [ ]:
# Define the Park class
class Park:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __repr__(self):
        return f"Park({', '.join(f'{k}={v}' for k, v in vars(self).items())})"